# Dependancy Installations

In [ ]:
!pip install -U sentence-transformers

     |████████████████████████████████| 85 kB 2.6 MB/s 
     |████████████████████████████████| 2.6 MB 36.9 MB/s 
     |████████████████████████████████| 1.2 MB 32.5 MB/s 
     |████████████████████████████████| 50 kB 6.3 MB/s 
     |████████████████████████████████| 636 kB 51.2 MB/s 
     |████████████████████████████████| 895 kB 53.3 MB/s 
     |████████████████████████████████| 3.3 MB 32.8 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.0.0-py3-none-any.whl size=126710 sha256=0719a6f81c5f331157d056927b7ecdefbb7a56491998929e0c4c8147baf0107b
  Stored in directory: /root/.cache/pip/wheels/d1/c1/0f/faafd427f705c4b012274ba60d9a91d75830306811e1355293
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
!pip install faiss-gpu

     |████████████████████████████████| 89.7 MB 10 kB/s 


In [ ]:
import faiss,numpy

In [ ]:
from sentence_transformers import SentenceTransformer
embed_model = SentenceTransformer('stsb-roberta-base-v2')

Downloading:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.68k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/675 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
import spacy
import nltk
import re

In [ ]:
import pandas as pd

# Setting Variables

In [ ]:
job_file_path='/content/Jobs.csv'
candidate_file_path='/content/Candidate.csv'

In [ ]:
# Relevant columns for job dataframe
job_responsibility_column=8
job_description_column=3
job_qualification_column=9

In [ ]:
# Relevant columns for candidate dataframe
candidate_experience_column=5
candidate_education_column=4
candidate_skills_column=9
candidate_certificates_column=11

In [ ]:
# Setting Faiss variables
d = 768                   # dimension of vectors
nb = 1585                 # database size (jobs+candidates)
nq = 1                    # nb of queries
k = 1585                 # To see 1000 nearest neighbors

In [ ]:
n_jobs=10                 # number of similar jobs to be retrieved
n_candidates=10           # number of similar candidates to be retrieved

# Data Loading

In [ ]:
jobs_df=pd.read_csv(job_file_path,encoding = "ISO-8859-1")
cand_df=pd.read_csv(candidate_file_path,encoding = "ISO-8859-1")
del jobs_df['Date']                   # Since contained all NAN values
del jobs_df['Job ID']                 # Since contained all NAN values
jobs_df=jobs_df.dropna()              # Dropping further NAN values

In [ ]:
jobs_df=jobs_df.drop_duplicates(keep='first',subset='Description')    # Dropping duplicates
cand_df=cand_df.drop_duplicates(keep='first',subset='Education')      # Dropping duplicates

In [ ]:
# Concatentaing text from desired column i.e responsibility, description and qualification
job_text=[]
job_ids=[]
for i in range(len(jobs_df)):
  concatenated_text=(jobs_df.iloc[i,job_responsibility_column]+ '. '+ \
                     jobs_df.iloc[i,job_description_column]+'. '+ \
                     jobs_df.iloc[i,job_qualification_column]).replace('\n','. ')
  text=concatenated_text.replace('\n\n','')
  job_text.append(text)
  job_ids.append(i)


# Concatentaing text from desired column i.e experience, education,skills and certficates
candidate_text=[]
cand_ids=[]
for i in range(len(cand_df)):
  concatenated_text=str(cand_df.iloc[i,candidate_experience_column])+ '. '+str(cand_df.iloc[i,candidate_education_column])+ \
  '. '+str(cand_df.iloc[i,candidate_skills_column])+'. '+str(cand_df.iloc[i,candidate_certificates_column]).replace('\n','. ')
  text=concatenated_text.replace('\n\n','')
  text=text.replace('\n','')
  candidate_text.append(text)
  cand_ids.append(i)
jobs_df['job_text']=job_text
#jobs_df['job_text']=job_text
cand_df['candidate_text']=candidate_text
#cand_df['job_text']=job_text

In [ ]:
# Creating set to handle duplicates, since data has lot of duplicates
complete_text=job_text+candidate_text

In [ ]:
# Create mapping to map back to original dataframe
indexes=[i for i in range(len(complete_text))]
mapping=dict(zip(indexes,(job_ids+cand_ids)))

In [ ]:
# code to extract number of professional years from candidate experience
import re
pattern=re.compile(r'\d [y]r')
years=[]
for i in range(len(cand_df)):
  try:
    years.append(pattern.findall(cand_df.iloc[i,5])[0])
  except:
    years.append('NA')
cand_df['Duration']=years

In [ ]:
cand_df

,Unnamed: 0,ID,About,Accomplishments,Education,Experience,Industry,Level,Location,Skills,Type,Certification,candidate_text,Duration
0,0,1,\n\n About\n \n\n\n\n Harnessing my str...,NaN,\n\n \n\n\n\n\nQassim University\n\nDegree Nam...,NaN,NaN,Mid Senior level,Riyadh,\n\n\n\n\n \n Communication\n ...,Full-time,\n\n \n\n\n\n\n Technical Tools\n \n\n\n ...,nan. Qassim UniversityDegree NameBachelor's d...,NA
1,1,2,\n\n About\n \n\n\n\n Junior experience...,NaN,\n\n \n\n\n\n\nKing Faisal University\n\nDegre...,\n\n \n\n\n\nAccountant\nCompany Name\n\n ...,NaN,Entry level,Riyadh,\n\n\n\n\n\n SMACC (School of Managme...,Full-time,NaN,AccountantCompany Name Zahra Breast Ca...,1 yr
2,2,3,\n\n About\n \n\n,CPR,\n\n \n\n\n\n\nSyddansk Universitet - Universi...,\n\n \n\n\n\nResearch Coordinator\nCompany N...,NaN,Entry level,Riyadh,\n\n\n\n\n \n data analysis\n ...,Full-time,\n\n \n\n\n\n\n NIH\n \n\n\n Issuin...,Research CoordinatorCompany Name Speci...,1 yr
3,3,4,"\n\n About\n \n\n\n\n Hard worker "" ver...",NaN,\n\n\n\n\n\n\n\nSouth tyneside college \n\nDeg...,\n\n\n\n\n\n\nMechanic\nCompany Name\n\n ...,NaN,Entry level,Riyadh,\n\n\n\n\n\n Microsoft Office\n ...,Full-time,NaN,MechanicCompany Name Seadrill Gcc Opera...,2 yr
4,4,5,\n\n About\n \n\n\n\n Always seeking fo...,Farsi,\n\n \n\n\n\n\nArab Open University - Saudi\n\...,\n\n\n\n\n\n\nProgrammer and Web Developer\n...,NaN,Entry level,Riyadh,\n\n\n\n\n\n Programming\n \n ...,Full-time,\n\n \n\n\n\n\n English Certificate\n \n\...,Programmer and Web DeveloperCompany Name ...,NA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1611,1611,1612,\n\n About\n \n\n\n\n As a recruitment ...,Arabic,\n\n \n\n\n\n\nAnwar Ul-Uloom College of Educa...,\n\n \n\n\n\nRecruitment Consultant\nCompany...,NaN,Entry level,Riyadh,\n\n\n\n\n \n Computer Skills\n ...,Full-time,NaN,Recruitment ConsultantCompany Name Ant...,4 yr
1612,1612,1613,\n\n About\n \n\n\n\n Objective:Working...,English,\n\n \n\n\n\n\nArab Open University - Saudi\n\...,\n\n \n\n\n\nSenior Human Resources Speciali...,NaN,Mid Senior level,Riyadh,\n\n\n\n\n \n Human Resources\n ...,Full-time,\n\n \n\n\n\n\n Certified Talent Management...,Senior Human Resources SpecialistCompany Na...,4 yr
1613,1613,1614,\n\n About\n \n\n\n\n - Languages : Ara...,Ù ÙØµØ© Ø£Ù Ø§Ù Ø§ÙÙØ§ÙÙÙÙØ©,\n\n\n\n\n\n\n\nArab Open University\n\nDegree...,\n\n\n\n\n\n\nCo-Founder and Chief Executive...,NaN,Entry level,Riyadh,\n\n\n\n\n \n Mobile Applications\n...,Full-time,\n\n \n\n\n\n\n OpenText Content Server Adm...,Co-Founder and Chief Executive OfficerCompan...,1 yr
1614,1614,1615,\n\n About\n \n\n\n\n Seeking a new cha...,NaN,\n\n\n\n\n\n\n\nAOU (Arab Open University)\n\n...,\n \n\n\n\n\n\nCompany Name\nThe National Co...,NaN,Mid Senior level,Riyadh,\n\n\n\n\n \n Team Leadership\n ...,Full-time,NaN,Company NameThe National Commercial Bank - ...,6 yr


# Data Cleaning

In [ ]:
# Data preprocessing function
def preprocess(sentence):
  sentence = "".join(filter(lambda x: not x.isdigit(), sentence))
  sentence = ' '.join([i for i in sentence.split() if '@' not in i])
  sentence=re.sub(' +', ' ', sentence)
  sentence=' '.join([item for item in sentence.split() if '@' not in item])
  sentence = ' '.join([item for item in sentence.split() if len(item)>2])
  sentence=re.sub("[\(\[]*?[\)\]]", "", sentence)
  sentence = re.sub(r"[^a-zA-Z0-9.,]+", ' ', sentence)
  sentence=sentence.rstrip("\n")
  sentence=sentence.strip()
  strText=sentence.split()
  sentence=" ".join(strText)
  sentence=sentence.lower()
  return sentence

In [ ]:
# Creating list of cleaned text, using above preprocessing function
cleaned_complete=[]
for i in range(len(complete_text)):
  cleaned_complete.append(preprocess(complete_text[i]))

# Generating embeddings and FAISS indexing

In [ ]:
# Creating embeddings 
cleaned_embed=embed_model.encode(cleaned_complete,show_progress_bar=True)

Batches:   0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
import numpy as np
np.random.seed(1234)             # make reproducible
res = faiss.StandardGpuResources()  # use a single GPU
index_flat = faiss.IndexFlatL2(d)  # build a flat (CPU) index
gpu_index_flat = faiss.index_cpu_to_gpu(res, 0, index_flat)
gpu_index_flat.add(cleaned_embed)         # add vectors to the index
print(gpu_index_flat.ntotal)

1585


In [ ]:
# Creating distance and index array, here k refers to number of nearest neighbors to be searched
D, I = gpu_index_flat.search(cleaned_embed, k) 

In [ ]:
act=[]
for ele in I[4]:
  if ele > 163:
    act.append(ele)
act[:5]


[639, 424, 343, 1061, 1282]

In [ ]:
cleaned_complete[4]

'non. ideal candidate should available for local transfer kingdom saudi arabia.. bachelor s degree equivalent experience.. strong english verbal, written, and organizational skills.. years experience document controller, having exposure mechanical, electrical, civil any construction discipline.. flexible any location within k.s.a.. transferable iqama'

# Extracting similar candidates for a given job

In [ ]:
'''
Extracts 10 most similar candidates for a given job along with distance scores
'''
ids=[]
retrieved_candidates=[]
distance=[]
cand_index=[]
job=[]
for i in range(163):
  act=[]
  dist=[]
  ids=[]
  for ele,el in (zip(I[i],D[i])):
    if ele> 163:
      act.append(complete_text[ele])
      dist.append(el)
      ids.append(mapping[ele])
  job.append(complete_text[i])
  retrieved_candidates.append(act[:10])
  distance.append(dist[:10])
  cand_index.append(ids[:10])

In [ ]:
jobs_dataframe=pd.DataFrame(list(zip(job,cand_index,retrieved_candidates,distance)),columns=['Job Description','Candidate ID','Retrieved Candidates','Distance Score'])

In [ ]:
jobs_dataframe

,Job Description,Candidate ID,Retrieved Candidates,Distance Score
0,Keep abreast of latest industry trends to ensu...,"[1039, 89, 1304, 1320, 827, 979, 406, 593, 141...",[ Senior Director of Business DevelopmentCom...,"[185.21564, 187.28, 189.7872, 190.01468, 191.5..."
1,Project Planning. . Contribute to the developm...,"[424, 332, 684, 130, 462, 1390, 460, 593, 979,...",[ IT PMO HeadCompany Name KAFD Developm...,"[122.37575, 145.19763, 146.51833, 161.88748, 1..."
2,Project Planning. Contribute to the developmen...,"[424, 332, 684, 130, 462, 1390, 460, 593, 979,...",[ IT PMO HeadCompany Name KAFD Developm...,"[122.37575, 145.19763, 146.51833, 161.88748, 1..."
3,Participate in the design and implementationof...,"[1039, 608, 424, 332, 562, 130, 827, 811, 509,...",[ Senior Director of Business DevelopmentCom...,"[177.41882, 178.11926, 182.01758, 182.4215, 19..."
4,non. Ideal candidate s should be available for...,"[476, 261, 180, 898, 1119, 360, 1068, 770, 111...",[ TranslatorCompany Name Saudi Air Navi...,"[179.0364, 190.90985, 193.63956, 195.6012, 196..."
...,...,...,...,...
158,You will be required to perform the following:...,"[460, 1018, 1369, 561, 829, 1139, 23, 1182, 11...",[ Company NameLeading Machine Learning FirmTo...,"[190.0947, 190.62396, 192.89499, 194.6658, 196..."
159,Own and operationalize Tasnee proprietary Dyna...,"[460, 657, 340, 593, 691, 1215, 1254, 1116, 36...",[ Company NameLeading Machine Learning FirmTo...,"[177.76233, 207.37767, 214.19357, 214.46242, 2..."
160,Plan and implement a 12-week curriculum in con...,"[978, 993, 1015, 1182, 1139, 574, 1164, 1321, ...",[ Company NameMultipleTotal Duration11 yrs Ti...,"[147.71123, 165.68971, 179.66693, 179.77647, 1..."
161,Design and develop statistical & machine learn...,"[460, 688, 292, 104, 1168, 1131, 657, 337, 126...",[ Company NameLeading Machine Learning FirmTo...,"[195.75018, 219.19302, 219.25308, 243.24588, 2..."


In [ ]:
# Writing output dataframe as json file
jobs_dataframe.to_csv('job_dataframe.csv')

# Extracting similar jobs for a given candidate

In [ ]:
retrieved_jobs=[]
distance=[]
candidates=[]
for i in range(163,len(complete_text)):
  act=[]
  dist=[]
  for ele,el in (zip(I[i],D[i])):
    if ele< 185:
      act.append(complete_text[ele])
      dist.append(el)
  candidates.append(complete_text[i])
  retrieved_jobs.append(act[:10])
  distance.append(dist[:10])

In [ ]:
candidate_dataframe=pd.DataFrame(list(zip(candidates,retrieved_jobs,distance)),columns=['Candidate Description','Retrieved Jobs','Distance Score'])

In [ ]:
# Writing output dataframe as json file
candidate_dataframe.to_csv('candidate_dataframe.csv')

In [ ]:
candidate_dataframe

,Candidate Description,Retrieved Jobs,Distance Score
0,nan. Qassim UniversityDegree NameBachelor's d...,[nan. Qassim UniversityDegree NameBachelor's ...,"[0.0, 142.6547, 155.5126, 172.30576, 178.61826..."
1,AccountantCompany Name Zahra Breast Ca...,[ AccountantCompany Name Zahra Breast C...,"[6.1035156e-05, 200.12787, 205.55994, 227.1391..."
2,Research CoordinatorCompany Name Speci...,[ Research CoordinatorCompany Name Spec...,"[0.0, 124.08557, 162.49597, 170.28125, 175.006..."
3,MechanicCompany Name Seadrill Gcc Opera...,[ MechanicCompany Name Seadrill Gcc Oper...,"[3.0517578e-05, 194.70508, 221.50195, 224.2464..."
4,Programmer and Web DeveloperCompany Name ...,[ Programmer and Web DeveloperCompany Name ...,"[0.0, 175.43951, 197.95471, 217.29611, 217.418..."
...,...,...,...
1417,Recruitment ConsultantCompany Name Ant...,[To execute overall business development initi...,"[173.97716, 174.08252, 174.42258, 183.1286, 18..."
1418,Senior Human Resources SpecialistCompany Na...,[ QA/QC EngineerCompany Name SWME LLC (...,"[240.4018, 254.86668, 262.934, 262.934, 263.54..."
1419,Co-Founder and Chief Executive OfficerCompan...,[ Programmer and Web DeveloperCompany Name ...,"[172.06018, 187.53644, 188.86087, 189.88742, 1..."
1420,Company NameThe National Commercial Bank - ...,[As a member of the executive team participate...,"[212.56538, 213.24821, 218.66725, 223.99825, 2..."
